In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import numpy as np
import config, torch
import os
from derma.dataset import Derma, get_samples_weight

In [ ]:
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip, RandomVerticalFlip, CenterCrop
transform = Compose([
        RandomHorizontalFlip(),
        RandomVerticalFlip(),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
dataset = Derma(config.DATASET_DIR, transform=ToTensor())

from torch.utils.data import DataLoader, random_split


val_size = int(0.1*len(dataset))
train_size = len(dataset) - val_size

train_set, val_set = random_split(dataset, [train_size, val_size])
train_sampler, _ = get_samples_weight(train_set)
train_loader = DataLoader(train_set, batch_size=128, shuffle=False, sampler=train_sampler)

val_sampler, _ = get_samples_weight(val_set)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False, sampler=val_sampler)

In [ ]:
from derma.architecture import InvertedResidual
from torchvision.models import MobileNetV2

labels = [0, 1]
model = MobileNetV2(num_classes=len(labels), block=InvertedResidual)

In [ ]:
from derma.utils import train
from torch.utils.tensorboard import SummaryWriter

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
tb_writer = SummaryWriter(log_dir='test')
criterion = torch.nn.CrossEntropyLoss()

train(model, [train_loader, val_loader], optimizer, criterion, 10, tb_writer)